In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
KEY=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [42]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-Nemo-Instruct-2407",
    task="text-generation",
    max_new_tokens=1024,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\viraj\.cache\huggingface\token
Login successful


In [43]:
llm

HuggingFaceEndpoint(repo_id='mistralai/Mistral-Nemo-Instruct-2407', max_new_tokens=1024, repetition_penalty=1.03, model='mistralai/Mistral-Nemo-Instruct-2407', client=<InferenceClient(model='mistralai/Mistral-Nemo-Instruct-2407', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-Nemo-Instruct-2407', timeout=120)>, task='text-generation')

In [44]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [45]:
RESPONSE_JSON ={
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    
}

In [46]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not  repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}


"""

In [47]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [48]:
quiz_chain = LLMChain(llm=chat_model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [49]:
TEMPLATE2="""
You are an expert english gramarian and writer. Given a Mutiple Choice Quiz fro {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not as per with the cognitive and analytical ability of the student,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the ability of the student.
Quiz_MCQ:
{quiz}

check from an expert English Writer of the above quiz    

"""

In [50]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [51]:
review_chain = LLMChain(llm=chat_model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [52]:
generate_evaluation_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                           output_variables=["quiz", "review"], verbose=True)


In [53]:
file_path = r"C:\Users\viraj\Documents\MCQ-GEN using HUGGING_FACE\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()

In [54]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [64]:
NUMBER = 1
SUBJECT = "machine learning"
TONE = "simple"

In [65]:
# Run chain
response = generate_evaluation_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of 

In [66]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFro

In [67]:
quiz=response.get("quiz")

In [68]:
response.get(quiz)

In [70]:
quiz=json.loads(quiz)

In [73]:
quiz_tables_data = []
for key,value in quiz.items():
    mcq= value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
correct = value["correct"]
quiz_tables_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [74]:
quiz_tables_data

[{'MCQ': 'Which of the following is NOT a typical application field of machine learning?',
  'Choices': 'a: Natural Language Processing | b: Agriculture | c: Mathematics | d: Medicine',
  'Correct': 'c'}]

In [76]:
quiz=pd.DataFrame(quiz_tables_data)

In [77]:
quiz.to_csv("ML.csv",index=False)